In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
h2o.init()
h2o.no_progress()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 44 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_thinker_e4tyfa
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,690 Mb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [3]:
data = pd.read_csv('../data/ieee-fraud-detection/train_transaction.csv')
data

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
feature = data.drop('isFraud', axis=1)
feature

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
label = data.isFraud
label

0         0
1         0
2         0
3         0
4         0
         ..
590535    0
590536    0
590537    0
590538    0
590539    0
Name: isFraud, Length: 590540, dtype: int64

In [6]:
y = 'isFraud'
X = list(data.columns)

In [7]:
train, valid = train_test_split(
    data,
    test_size=0.2,
    random_state=42
)
display(train)
valid

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
5307,2992307,0,174911,59.000,W,4988,334.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191582,3178582,0,4301977,23.378,C,3867,296.0,185.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260168,3247168,0,6229929,35.950,W,12577,268.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18516,3005516,0,497176,100.000,R,6019,583.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47538,3034538,0,1124702,75.000,H,16075,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,3097268,0,2157580,25.000,H,16796,555.0,210.0,visa,137.0,...,0.0,50.0,50.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0
259178,3246178,0,6213562,357.950,W,16727,111.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365838,3352838,0,9076295,156.000,W,3283,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131932,3118932,0,2614452,59.000,W,9500,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
470624,3457624,0,12153579,724.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565820,3552820,0,15005886,108.500,W,12544,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284083,3271083,0,6970178,47.950,W,9400,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239689,3226689,0,5673658,100.599,C,15885,545.0,185.0,visa,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281855,3268855,0,6886780,107.950,W,15497,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373780,3360780,0,9314548,113.000,W,2242,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245203,3232203,0,5843908,117.000,W,8763,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450760,3437760,0,11491417,59.513,C,16136,204.0,185.0,visa,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529422,3516422,0,13923492,21.580,C,15885,545.0,185.0,visa,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
h2o_train = h2o.H2OFrame(train)
h2o_valid = h2o.H2OFrame(valid)

H2OServerError: HTTP 500 Server Error:
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
<title>Error 500 Server Error</title>
</head>
<body><h2>HTTP ERROR 500</h2>
<p>Problem accessing /3/PostFile. Reason:
<pre>    Server Error</pre></p><h3>Caused by:</h3><pre>java.lang.OutOfMemoryError: Java heap space
</pre>

</body>
</html>


In [ ]:
h2o_train[y] = h2o_train[y].asfactor() # Which Column is Label?
h2o_valid[y] = h2o_valid[y].asfactor()

In [ ]:
aml = H2OAutoML(
    max_runtime_secs=max_runtime_secs
)
aml.train(
    x=X,
    y=y,
    training_frame=h2o_train,
    leaderboard_frame=h2o_valid
)


21:46:43.16: AutoML: XGBoost is not available; skipping it.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_2_20220612_214641

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.016467450104500805
RMSE: 0.128325562942466
LogLoss: 0.07050513659513263
Null degrees of freedom: 10021
Residual degrees of freedom: 10019
Null deviance: 2969.2719500249727
Residual deviance: 1413.2049579128382
AIC: 1419.2049579128382
AUC: 0.9312026866106906
AUCPR: 0.7011529633312876
Gini: 0.8624053732213812

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.17219803893840444: 


,,0,1,Error,Rate
0,0,9565.0,117.0,0.0121,(117.0/9682.0)
1,1,118.0,222.0,0.3471,(118.0/340.0)
2,Total,9683.0,339.0,0.0234,(235.0/10022.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.172198,0.653903,190.0
1,max f2,0.101003,0.670467,229.0
2,max f0point5,0.586064,0.763359,95.0
3,max accuracy,0.385226,0.980842,123.0
4,max precision,0.993222,1.000000,0.0
5,max recall,0.006203,1.000000,388.0
6,max specificity,0.993222,1.000000,0.0
7,max absolute_mcc,0.385226,0.661901,123.0
8,max min_per_class_accuracy,0.025306,0.854266,318.0
9,max mean_per_class_accuracy,0.041582,0.874169,289.0



Gains/Lift Table: Avg response rate:  3.39 %, avg score:  3.60 %


...

,,0,1,Error,Rate
0,0,45461.0,313.0,0.0068,(313.0/45774.0)
1,1,907.0,747.0,0.5484,(907.0/1654.0)
2,Total,46368.0,1060.0,0.0257,(1220.0/47428.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.298726,0.550479,171.0
1,max f2,0.106425,0.535878,249.0
2,max f0point5,0.614227,0.679557,99.0
3,max accuracy,0.505117,0.976006,120.0
4,max precision,0.991552,1.000000,0.0
5,max recall,0.004324,1.000000,399.0
6,max specificity,0.991552,1.000000,0.0
7,max absolute_mcc,0.450387,0.561138,132.0
8,max min_per_class_accuracy,0.020550,0.799879,342.0
9,max mean_per_class_accuracy,0.032871,0.808425,320.0



Gains/Lift Table: Avg response rate:  3.49 %, avg score:  3.43 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010015,0.818176,27.225900,27.225900,0.949474,0.935273,0.949474,0.935273,0.272672,0.272672,2622.589957,2622.589957,0.272148
1,2,0.020009,0.351907,15.849744,21.543816,0.552743,0.580431,0.751317,0.758039,0.158404,0.431076,1484.974413,2054.381561,0.425920
2,3,0.030003,0.187646,6.291501,16.463284,0.219409,0.257824,0.574139,0.591418,0.062878,0.493954,529.150149,1546.328371,0.480715
3,4,0.040019,0.125572,4.889796,13.566863,0.170526,0.152498,0.473130,0.481572,0.048972,0.542926,388.979571,1256.686285,0.521080
4,5,0.050013,0.093589,3.690208,11.593197,0.128692,0.107850,0.404300,0.406891,0.036880,0.579807,269.020760,1059.319734,0.548937
5,6,0.100004,0.043451,2.092251,6.843726,0.072965,0.061640,0.238668,0.234302,0.104595,0.684401,109.225134,584.372592,0.605514
6,7,0.150017,0.028451,1.196795,4.961151,0.041737,0.034549,0.173015,0.167708,0.059855,0.744256,19.679514,396.115110,0.615712
7,8,0.200008,0.021880,0.810294,3.923656,0.028258,0.024875,0.136833,0.132007,0.040508,0.784764,-18.970613,292.365558,0.605885
8,9,0.300013,0.015110,0.634798,2.827370,0.022138,0.018244,0.098601,0.094086,0.063482,0.848247,-36.520210,182.736969,0.568044
9,10,0.399996,0.011814,0.489805,2.243071,0.017081,0.013040,0.078225,0.073828,0.048972,0.897219,-51.019550,124.307081,0.515190
